In [ ]:
""" import logging from pyspark.sql import functions as F from pyspark.sql.types import IntegerType from datetime import datetime  # Configure logging logging.basicConfig(level=logging.INFO) logger = logging.getLogger(__name__)  try:     # Load data from Unity Catalog tables     policy_df = spark.table("genai_demo.jnj.policy")     claims_df = spark.table("genai_demo.jnj.claims")     demographics_df = spark.table("genai_demo.jnj.demographics")     scores_df = spark.table("genai_demo.jnj.scores")     aiml_insights_df = spark.table("genai_demo.jnj.aiml_insights")      # Select necessary fields     demographics_df = demographics_df.select(         "Customer_ID", "Customer_Name", "Email", "Phone_Number", "Address", "City", "State",          "Postal_Code", "Date_of_Birth", "Gender", "Marital_Status", "Occupation",          "Income_Level", "Customer_Segment"     )          claims_df = claims_df.select(         "Claim_ID", "Policy_ID", "Claim_Date", "Claim_Type", "Claim_Status",          "Claim_Amount", "Claim_Payout"     )          policy_df = policy_df.select(         "Policy_ID", "Customer_ID", "Policy_Type", "Policy_Status", "Policy_Start_Date",          "Policy_End_Date", "Policy_Term", "Policy_Premium", "Total_Premium_Paid",          "Renewal_Status", "Policy_Addons"     )          scores_df = scores_df.select(         "Customer_ID", "Credit_Score", "Fraud_Score", "Customer_Risk_Score"     )          aiml_insights_df = aiml_insights_df.select(         "Customer_ID", "Churn_Probability", "Next_Best_Offer",          "Claims_Fraud_Probability", "Revenue_Potential"     )      # Join demographics and policy data on Customer_ID     customer_policy_df = demographics_df.join(policy_df, "Customer_ID", "inner")      # Join the result with claims data on Policy_ID     customer_policy_claims_df = customer_policy_df.join(claims_df, "Policy_ID", "inner")      # Aggregate data     summary_df = customer_policy_claims_df.groupBy("Customer_ID").agg(         F.count("Claim_ID").alias("Total_Claims"),         F.max("Claim_Date").alias("Recent_Claim_Date"),         F.avg("Claim_Amount").alias("Average_Claim_Amount"),         F.countDistinct("Policy_ID").alias("Policy_Count")     )      # Join summarized data with detailed customer data     detailed_customer_df = customer_policy_claims_df.join(summary_df, "Customer_ID", "inner")      # Custom Calculations     detailed_customer_df = detailed_customer_df.withColumn(         "Age", F.floor(F.datediff(F.current_date(), F.to_date("Date_of_Birth", "yyyy-MM-dd")) / 365.25)     ).withColumn(         "Claim_To_Premium_Ratio", F.when(F.col("Total_Premium_Paid") != 0, F.col("Claim_Amount") / F.col("Total_Premium_Paid")).otherwise(0)     ).withColumn(         "Claims_Per_Policy", F.when(F.col("Policy_Count") != 0, F.col("Total_Claims") / F.col("Policy_Count")).otherwise(0)     ).withColumn(         "Retention_Rate", F.lit(0.85)     ).withColumn(         "Cross_Sell_Opportunities", F.lit("Multi-Policy Discount, Home Coverage Add-on")     ).withColumn(         "Upsell_Potential", F.lit("Premium Vehicle Coverage")     )      # Join with AI/ML insights and scores data     final_df = detailed_customer_df.join(aiml_insights_df, "Customer_ID", "inner").join(scores_df, "Customer_ID", "inner")     final_df.show(10)     # Write the final output to Unity Catalog table     #spark.sql("DROP TABLE IF EXISTS genai_demo.jnj.final_customer_data")     #final_df.write.format("delta").mode("overwrite").saveAsTable("genai_demo.jnj.final_customer_data")      logger.info("Data processing and migration completed successfully.")  except Exception as e:     logger.error(f"An error occurred during data processing: {e}") """
